#### <u>Движение ХКУ</u>

$$ \ddot{\vec r} = \begin{bmatrix} -2\omega_0 \dot y \\ 3\omega_0^2 y + 2 \omega_0 \dot x \\ -\omega_0^2 z \end{bmatrix}$$

In [10]:
import sympy

w, t, tau = sympy.var('w_0 t tau', real=True, positive=True)
r0 = sympy.var('r^0_x r^0_y r^0_z')
v0 = sympy.var('v^0_x v^0_y v^0_z')
r = [sympy.Function('r_x'), sympy.Function('r_y'), sympy.Function('r_z')]
v = [sympy.Function('v_x'), sympy.Function('v_y'), sympy.Function('v_z')]

eqs = [sympy.Eq(r[0](t).diff(t), v[0](t)), 
       sympy.Eq(r[1](t).diff(t), v[1](t)), 
       sympy.Eq(r[2](t).diff(t), v[2](t)), 
       sympy.Eq(v[0](t).diff(t), -2*w*v[1](t)), 
       sympy.Eq(v[1](t).diff(t), 3*w**2*r[1](t) + 2*w*v[0](t)), 
       sympy.Eq(v[2](t).diff(t), -w**2*r[2](t))]

print(f"Уравнения ХКУ:")
for e in eqs:
    display(e)
    
ics={r[0](0): r0[0], r[1](0): r0[1], r[2](0): r0[2], 
     v[0](0): v0[0], v[1](0): v0[1], v[2](0): v0[2]}

Уравнения ХКУ:


Eq(Derivative(r_x(t), t), v_x(t))

Eq(Derivative(r_y(t), t), v_y(t))

Eq(Derivative(r_z(t), t), v_z(t))

Eq(Derivative(v_x(t), t), -2*w_0*v_y(t))

Eq(Derivative(v_y(t), t), 3*w_0**2*r_y(t) + 2*w_0*v_x(t))

Eq(Derivative(v_z(t), t), -w_0**2*r_z(t))

In [12]:
anw1 = sympy.dsolve(eqs, [r[0](t), r[1](t), r[2](t), v[0](t), v[1](t), v[2](t)], ics=ics)
for a in anw1:
    display(a) 

Eq(r_x(t), r^0_x - t*(6*r^0_y*w_0 + 3*v^0_x) + 2*v^0_y*cos(t*w_0)/w_0 - 2*v^0_y/w_0 + (6*r^0_y + 4*v^0_x/w_0)*sin(t*w_0))

Eq(r_y(t), 4*r^0_y + 2*v^0_x/w_0 + v^0_y*sin(t*w_0)/w_0 - (3*r^0_y + 2*v^0_x/w_0)*cos(t*w_0))

Eq(r_z(t), r^0_z*cos(t*w_0) + v^0_z*sin(t*w_0)/w_0)

Eq(v_x(t), -6*r^0_y*w_0 - 3*v^0_x - 2*v^0_y*sin(t*w_0) + (6*r^0_y*w_0 + 4*v^0_x)*cos(t*w_0))

Eq(v_y(t), v^0_y*cos(t*w_0) + (3*r^0_y*w_0 + 2*v^0_x)*sin(t*w_0))

Eq(v_z(t), -r^0_z*w_0*sin(t*w_0) + v^0_z*cos(t*w_0))

In [2]:
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

earth_radius = 6371e3
height = 400e3
orbit_radius = earth_radius + height
mu = 5.972e24 * 6.67408e-11
w0 = np.sqrt(mu / orbit_radius ** 3)

def a(r,v):
    return np.array([-2 * w0 * v[2] - 1e-6,  # атмосферное сопротивление
                     -w0 ** 2 * r[1],
                     3 * w0 ** 2 * r[2] + 2 * w0 * v[0]])

d = []
for i_PCB in range(3):
    r = np.matrix([0, 0, 0])
    v = np.matrix([-0.001, 0, 0.02]) * (1 + 0.1*i_PCB)

    t, dt = 10000, 0.1
    for i in range(int(t // dt)):
        v = np.vstack((v, v[-1,:] + a(np.array(r[-1,:])[0],np.array(v[-1,:])[0])*dt))
        r = np.vstack((r, r[-1,:] + v[-1,:]*dt))
        #if ((r[-1,0])**2 + (r[-1,1])**2 < 1e-5) and (t > 1e3):
        #    print(f"Скорость пролёта LOS: {np.linalg.norm(v[-1,:]):.5f} | {v[-1,:]}")

    d.append(go.Scatter3d(x=np.array(r[:,0].T)[0], mode='lines',
                          y=np.array(r[:,1].T)[0], line=dict(width=3),
                          z=np.array(r[:,2].T)[0], name="PCBsat orbit"))
    
        
d = [go.Scatter3d(x=[0,0], mode='lines', name="LOS", 
                  y=[0,0], line=dict(width=3),
                  z=[0, np.min(np.array(r[:,2].T)[0])])] + d

fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'surface'}, ]])
for i in d:
    fig.add_trace(i, row=1, col=1)
fig.update_layout(legend=dict(yanchor='top', xanchor='left', y=0.9, x=0.7))
fig.update_scenes(xaxis_title_text='x, m', xaxis_title_font_size=12, xaxis_tickfont_size=10,
                  yaxis_title_text='y, m', yaxis_title_font_size=12, yaxis_tickfont_size=10,
                  zaxis_title_text='z, m', zaxis_title_font_size=12, zaxis_tickfont_size=10,)
fig.show()


KeyboardInterrupt

